In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ann1-data/training_data/Species4/00266.jpg
/kaggle/input/ann1-data/training_data/Species4/00091.jpg
/kaggle/input/ann1-data/training_data/Species4/00296.jpg
/kaggle/input/ann1-data/training_data/Species4/00219.jpg
/kaggle/input/ann1-data/training_data/Species4/00130.jpg
/kaggle/input/ann1-data/training_data/Species4/00460.jpg
/kaggle/input/ann1-data/training_data/Species4/00254.jpg
/kaggle/input/ann1-data/training_data/Species4/00160.jpg
/kaggle/input/ann1-data/training_data/Species4/00109.jpg
/kaggle/input/ann1-data/training_data/Species4/00111.jpg
/kaggle/input/ann1-data/training_data/Species4/00056.jpg
/kaggle/input/ann1-data/training_data/Species4/00030.jpg
/kaggle/input/ann1-data/training_data/Species4/00340.jpg
/kaggle/input/ann1-data/training_data/Species4/00074.jpg
/kaggle/input/ann1-data/training_data/Species4/00009.jpg
/kaggle/input/ann1-data/training_data/Species4/00361.jpg
/kaggle/input/ann1-data/training_data/Species4/00308.jpg
/kaggle/input/ann1-data/trainin

In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.6.4


In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input 

train_data_dir = "/kaggle/input/ann1-data/training_data"

img_height = 96
img_width = 96
batch_size = 8


#for augmentation check parameters of this function
train_datagen = ImageDataGenerator(                 
    rotation_range=30,
    height_shift_range=50,
    width_shift_range=50,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True, 
    fill_mode='reflect',
preprocessing_function=preprocess_input, data_format=None, validation_split=0.16
) # set validation split

#train_datagen.fit('/kaggle/input/ann1-data/training_data/Species1/00000.jpg')

train_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=batch_size,
    seed=seed,
    shuffle=True,
    class_mode='categorical',
    subset='training') # set as training data

valid_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
    validation_split=0.2) # set validation split

valid_gen = valid_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=batch_size,
    seed=seed,
    shuffle=False,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 2980 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


In [5]:
input_shape = (96,96,3)

from tensorflow.keras.applications import VGG16 
from tensorflow.keras.regularizers import l2

base_net = VGG16(include_top=False, weights="imagenet", input_shape=input_shape)
base_net.trainable = False

inputs = tfk.Input(shape=input_shape)
x = base_net(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    160, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed),
    kernel_regularizer=l2(l=0.03))(x)
x = tfkl.Dense(
    224, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed),
    kernel_regularizer=l2(l=0.03))(x)
x = tfkl.Dense(
    64, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed),
    kernel_regularizer=l2(l=0.03))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
      8, 
      activation='softmax',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

for layer in base_net.layers[:-10]:
        layer.trainable = False

      # Connect input and output through the Model class
model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

      # Compile the model
    


2022-11-21 15:34:24.031247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 15:34:24.119787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 15:34:24.120616: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 15:34:24.121800: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 3s 0us/step


In [6]:
# Train the model
tl_history = model.fit(
    x = train_gen,
    batch_size = 256,
    epochs = 100,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

2022-11-21 15:34:30.251622: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-11-21 15:34:31.826091: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


373/373 [==============================] - 29s 60ms/step - loss: 16.2440 - accuracy: 0.2705 - val_loss: 9.9838 - val_accuracy: 0.4136
Epoch 2/100
373/373 [==============================] - 12s 32ms/step - loss: 7.2056 - accuracy: 0.3611 - val_loss: 4.9605 - val_accuracy: 0.4348
Epoch 3/100
373/373 [==============================] - 12s 33ms/step - loss: 3.9667 - accuracy: 0.3872 - val_loss: 3.0671 - val_accuracy: 0.4618
Epoch 4/100
373/373 [==============================] - 12s 33ms/step - loss: 2.7078 - accuracy: 0.4584 - val_loss: 2.2031 - val_accuracy: 0.5085
Epoch 5/100
373/373 [==============================] - 12s 33ms/step - loss: 2.1731 - accuracy: 0.4802 - val_loss: 1.9930 - val_accuracy: 0.4674
Epoch 6/100
373/373 [==============================] - 12s 33ms/step - loss: 1.9091 - accuracy: 0.4977 - val_loss: 1.6634 - val_accuracy: 0.5411
Epoch 7/100
373/373 [==============================] - 13s 34ms/step - loss: 1.7862 - accuracy: 0.5144 - val_loss: 1.6121 - val_accuracy: 0.5

In [7]:
# Freeze first N layers, e.g., until 14th
model.get_layer('vgg16').trainable = True
for i, layer in enumerate(model.get_layer('vgg16').layers):
   print(i, layer.name, layer.trainable)
for i, layer in enumerate(model.get_layer('vgg16').layers[:12]):
  layer.trainable=False
for i, layer in enumerate(model.get_layer('vgg16').layers):
   print(i, layer.name, layer.trainable)
model.summary()

0 input_1 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_pool True
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________

In [8]:
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [9]:

# Fine-tune the model
ft_history = model.fit(
    x = train_gen,
    batch_size = 256,
    epochs = 100,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

Epoch 1/100
373/373 [==============================] - 14s 35ms/step - loss: 1.5902 - accuracy: 0.4893 - val_loss: 1.3132 - val_accuracy: 0.6020
Epoch 2/100
373/373 [==============================] - 13s 35ms/step - loss: 1.3426 - accuracy: 0.5946 - val_loss: 1.1125 - val_accuracy: 0.6331
Epoch 3/100
373/373 [==============================] - 13s 35ms/step - loss: 1.2333 - accuracy: 0.6215 - val_loss: 1.2064 - val_accuracy: 0.6048
Epoch 4/100
373/373 [==============================] - 13s 35ms/step - loss: 1.1520 - accuracy: 0.6409 - val_loss: 1.1124 - val_accuracy: 0.6133
Epoch 5/100
373/373 [==============================] - 14s 36ms/step - loss: 1.1304 - accuracy: 0.6513 - val_loss: 1.1520 - val_accuracy: 0.6317
Epoch 6/100
373/373 [==============================] - 13s 34ms/step - loss: 1.0668 - accuracy: 0.6567 - val_loss: 0.9483 - val_accuracy: 0.6714
Epoch 7/100
373/373 [==============================] - 13s 35ms/step - loss: 1.0082 - accuracy: 0.6903 - val_loss: 1.0670 - val_ac

In [10]:
# Freeze first N layers, e.g., until 14th
model.get_layer('vgg16').trainable = True
for i, layer in enumerate(model.get_layer('vgg16').layers):
   print(i, layer.name, layer.trainable)
for i, layer in enumerate(model.get_layer('vgg16').layers[:13]):
  layer.trainable=False
for i, layer in enumerate(model.get_layer('vgg16').layers):
   print(i, layer.name, layer.trainable)
model.summary()

ft_history = model.fit(
    x = train_gen,
    batch_size = 256,
    epochs = 100,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

0 input_1 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_pool True
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 3)]       0         
________________